In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
from datahelper import PostDataset, CommentDataset, get_indices
from torch.utils.data import DataLoader
from models import FineTuneBaseModel, FineTuneClassifier
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 6, 14, 14, 31, 50),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [7]:
print(len(db.query("SELECT COMMENT_ID FROM COMMENTS WHERE TARGET IN (0,1)")))

505


In [8]:
len(db.query('''SELECT TITLE, TARGET FROM POSTS WHERE TARGET IN (0,1);'''))

38

In [8]:
len(db.query('''SELECT COMMENT, TARGET FROM COMMENTS WHERE TARGET IN (0,1);'''))

203

In [7]:
db.query("SELECT * FROM STOCKS WHERE STOCK_ID='AMC' AND LAST_SCRAPED >= DATE_SUB(NOW(),INTERVAL 55 MINUTE)")

[('AMC', datetime.datetime(2021, 6, 6, 22, 15, 44))]

In [16]:
db.query("SELECT POST_ID FROM POSTS WHERE STOCK_ID='{}' AND LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = '{}'), INTERVAL {} HOUR);".format('BB', 'BB', 6))



[('nx2ihx',),
 ('nxcb5h',),
 ('nxp5r6',),
 ('nxtjj4',),
 ('nxv9jr',),
 ('nxx9sz',),
 ('ny01wx',),
 ('nz6tgd',),
 ('nzf0mj',),
 ('nzjcfg',)]

In [12]:
db.query("SELECT POST_ID FROM POSTS WHERE STOCK_ID='{}' AND LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = '{}'),INTERVAL {} HOUR);".format('BB','BB',6))



[('nx2ihx',),
 ('nxcb5h',),
 ('nxp5r6',),
 ('nxtjj4',),
 ('nxv9jr',),
 ('nxx9sz',),
 ('ny01wx',),
 ('nz6tgd',),
 ('nzf0mj',),
 ('nzjcfg',)]

In [4]:
db.query("SELECT LAST_SCRAPED FROM POSTS WHERE LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = 'GME'),INTERVAL 6 HOUR);")
         
         
         


[(datetime.datetime(2021, 6, 7, 1, 40, 2),),
 (datetime.datetime(2021, 6, 7, 1, 40, 2),),
 (datetime.datetime(2021, 6, 7, 1, 40, 2),),
 (datetime.datetime(2021, 6, 7, 1, 40, 2),),
 (datetime.datetime(2021, 6, 7, 1, 40, 2),),
 (datetime.datetime(2021, 6, 7, 1, 40, 2),),
 (datetime.datetime(2021, 6, 7, 1, 40, 2),),
 (datetime.datetime(2021, 6, 7, 1, 40, 2),),
 (datetime.datetime(2021, 6, 7, 1, 40, 1),),
 (datetime.datetime(2021, 6, 7, 1, 40, 1),),
 (datetime.datetime(2021, 6, 7, 1, 40),),
 (datetime.datetime(2021, 6, 7, 1, 40),),
 (datetime.datetime(2021, 6, 7, 1, 40, 1),),
 (datetime.datetime(2021, 6, 7, 1, 40, 1),),
 (datetime.datetime(2021, 6, 7, 1, 40),),
 (datetime.datetime(2021, 6, 7, 1, 40, 1),),
 (datetime.datetime(2021, 6, 7, 1, 40, 1),),
 (datetime.datetime(2021, 6, 7, 1, 40, 1),),
 (datetime.datetime(2021, 6, 7, 1, 40),),
 (datetime.datetime(2021, 6, 7, 1, 40),),
 (datetime.datetime(2021, 6, 7, 1, 40),),
 (datetime.datetime(2021, 6, 7, 1, 39, 59),),
 (datetime.datetime(2021, 6

In [11]:
db.query("SELECT LAST_SCRAPED FROM POSTS")

[(datetime.datetime(2021, 6, 6, 17, 24, 2),),
 (datetime.datetime(2021, 6, 6, 17, 24, 2),),
 (datetime.datetime(2021, 6, 6, 17, 24, 2),),
 (datetime.datetime(2021, 6, 6, 17, 24, 2),),
 (datetime.datetime(2021, 6, 6, 17, 24, 2),),
 (datetime.datetime(2021, 6, 6, 17, 24, 2),),
 (datetime.datetime(2021, 6, 6, 17, 24, 2),),
 (datetime.datetime(2021, 6, 6, 17, 24, 2),),
 (datetime.datetime(2021, 6, 6, 17, 24, 1),),
 (datetime.datetime(2021, 6, 6, 17, 24, 1),),
 (datetime.datetime(2021, 6, 6, 17, 24, 1),)]

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [4]:
db = Database()
db.use_database('DB1')
#db.drop_table("LABELED_POSTS")
#db.drop_table("LABELED_COMMENTS")

db.initialize_tables()

scrapewsb = ScrapeWSB('GME', 11, 2)

df = scrapewsb.scrape()
scrapewsb.convert(df)
print("DONE")

connection established
[(datetime.datetime(2021, 6, 6, 17, 23, 59),)]
connection established
[(datetime.datetime(2021, 6, 6, 17, 24),)]


  0%|          | 0/11 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 6, 6, 17, 24, 3),)]


100%|██████████| 11/11 [01:08<00:00,  6.24s/it]

DONE


In [4]:
db = Database()
db.use_database('DB1')

connection established
[(datetime.datetime(2021, 5, 30, 0, 20, 6),)]


In [3]:
db.drop_table('COMMENTS')
db.drop_table("POSTS")
db.drop_table("STOCKS")

In [3]:
indices = get_indices("GME")

connection established
[(datetime.datetime(2021, 6, 10, 20, 15, 50),)]


In [4]:
post_train = PostDataset(512, indices['post_train'])



In [5]:
post_trainloader = DataLoader(
    dataset=post_train, 
    batch_size=3, 
    num_workers=0,
    shuffle=True
    )

In [6]:
sample = iter(post_trainloader).next()

connection established
[(datetime.datetime(2021, 6, 10, 20, 15, 57),)]


/Users/ianleefmans/opt/anaconda3/envs/StockSentiment/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


connection established
[(datetime.datetime(2021, 6, 10, 20, 15, 58),)]
connection established
[(datetime.datetime(2021, 6, 10, 20, 15, 58),)]


In [7]:
sample


{'post': ['AMC YOLO \U0001f9a7🎯🚀🔥',
  'First ever gains post.. GME and AMC to the moon we hodl 🦍💎🙌🚀🌝',
  'AMC YOLO LFG!'],
 'post_input_ids': tensor([[  101, 24810,   162,  ...,     0,     0,     0],
         [  101,  1752,  1518,  ...,     0,     0,     0],
         [  101, 24810,   162,  ...,     0,     0,     0]]),
 'post_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([1, 1, 1])}

In [10]:
model = FineTuneBaseModel()

out = model(input_ids=sample['post_input_ids'], attention_masks=sample['post_attention_mask'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
out

tensor([[0.4842, 0.5158],
        [0.5731, 0.4269],
        [0.3851, 0.6149]], grad_fn=<SoftmaxBackward>)

In [8]:
model = FineTuneClassifier()
out = model(input_ids=sample['post_input_ids'], attention_masks=sample['post_attention_mask'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [9]:
out

SequenceClassifierOutput(loss=None, logits=tensor([[0.2113, 0.7564],
        [0.1775, 0.7651],
        [0.2064, 0.8715]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)